In [1]:
import numpy as np
import pandas as pd

In [2]:
#Parameters
num_variations = 3
slots = ['Mon_07:00', 'Mon_08:00', 'Tue_07:00', 'Tue_08:00']
excel_path = 'weight-ekfis-test.xlsx'  # Update this path as needed

In [3]:
def read_excel_data(file_path):
    try:
        # Read Excel file with multiple sheets (one per group)
        xl = pd.ExcelFile(file_path)
        group_data_dict = {}
        for sheet_name in xl.sheet_names:
            # First check if A1 is empty by reading just that cell
            check_a1 = pd.read_excel(xl, 
                                   sheet_name=sheet_name, 
                                   nrows=1,
                                   usecols=[0],
                                   keep_default_na=True)  # Keep NaN values
            
            # Skip this sheet if A1 is empty or NaN
            if check_a1.empty or pd.isna(check_a1.iloc[0, 0]) or check_a1.iloc[0, 0] == '':
                print(f"Skipping sheet {sheet_name} due to empty A1")
                continue
                
            # Read the full range if A1 is not empty
            df = pd.read_excel(xl, 
                             sheet_name=sheet_name, 
                             usecols="A:F",    # Columns A through F
                             nrows=11)         # First 11 rows
            
            # Ensure the DataFrame has the expected structure
            if 'Time' in df.columns:
                group_data_dict[sheet_name] = df
                
        return group_data_dict if group_data_dict else None
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return None

def create_sample_data(excel_file=None):
    # Try to read Excel data first
    excel_data = read_excel_data(excel_file) if excel_file else None
    
    group_data_dict = {}
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
    
    if excel_data:
        # Process Excel data when available
        for group_name, df in excel_data.items():
            # Ensure the DataFrame has the required columns
            if all(day in df.columns for day in days) and 'Time' in df.columns:
                # Convert time format if needed
                df['Time'] = df['Time'].apply(lambda x: f"{int(x):02d}:00" if isinstance(x, (int, float)) else x)
                group_data_dict[group_name] = df[['Time'] + days]
    return group_data_dict

# Use sample data with optional Excel input
sample_data = create_sample_data(excel_file=excel_path)

# Define groups and slots
groups = list(sample_data.keys())

def generate_random_weights(base_weights, num_variations=5):
    # Create multiple variations of the weights matrix
    variations = []
    for _ in range(num_variations):
        # Add random noise of order 2 to the base weights
        noise = np.random.uniform(-2, 2, base_weights.shape)
        # Ensure weights remain positive by taking absolute value
        new_weights = np.abs(base_weights + noise)
        variations.append(new_weights)
    return variations

# Base weights matrix
# print(sample_data)
base_weights = np.array([
    [sample_data[g].loc[
        sample_data[g].Time == s.split('_')[1],
        s.split('_')[0]
    ].values[0] for s in slots
    ] for g in groups
])

def gale_shapley_matching(weights):
    # Convert weights to preferences
    preferences = np.argsort(weights, axis=1)
    n_groups, n_slots = weights.shape
    
    # Initialize all groups and slots as free
    free_groups = set(range(n_groups))
    matches = [-1] * n_slots
    group_proposals = [0] * n_groups
    
    while free_groups:
        g = free_groups.pop()
        # Check if group has already proposed to all slots
        if group_proposals[g] >= n_slots:
            continue
        
        s = preferences[g][group_proposals[g]]
        group_proposals[g] += 1
        
        if matches[s] == -1:
            matches[s] = g
        else:
            current_g = matches[s]
            if weights[current_g][s] > weights[g][s]:
                matches[s] = g
                free_groups.add(current_g)
            else:
                free_groups.add(g)
    
    return matches

Skipping sheet Group 30 due to empty A1
Skipping sheet Group 8 due to empty A1


In [4]:

# Generate multiple variations of weights
weight_variations = generate_random_weights(base_weights, num_variations)

# Run the algorithm for unrandomized weights
print("Original scenario:\n")
print("Weights matrix:")
print(" ", end=" ")  # Changed to match randomized format
for slot in slots:
    print(f"{slot[:2]}{slot[4:6]}", end=" ")  # Changed to match randomized format
print()
print(base_weights.round(2))
final_matches = gale_shapley_matching(base_weights)
print("\nMatching results:")
for slot_idx, group_idx in enumerate(final_matches):
    print(f"{slots[slot_idx]} -> {groups[group_idx]}")
print("\n" + "-"*40 + "\n")

# Run the algorithm for each variation
print("Randomized scenarios:\n")
for i, weights in enumerate(weight_variations, 1):
    print(f"Scenario {i}:")
    print("Weights matrix:")
    print(" ", end=" ")  # Changed to match randomized format
    for slot in slots:
        print(f"{slot[:2]}{slot[4:6]}", end=" ")  # Changed to match randomized format
    print()
    print(weights.round(2))
    final_matches = gale_shapley_matching(weights)
    print("\nMatching results:")
    for slot_idx, group_idx in enumerate(final_matches):
        print(f"{slots[slot_idx]} -> {groups[group_idx]}")
    print("\n" + "-"*40 + "\n")

Original scenario:

Weights matrix:
  Mo07 Mo08 Tu07 Tu08 
[[5. 4. 4. 8.]
 [2. 2. 3. 1.]
 [3. 9. 6. 8.]
 [9. 5. 7. 5.]
 [6. 4. 8. 2.]
 [4. 4. 5. 7.]]

Matching results:
Mon_07:00 -> Group 3
Mon_08:00 -> Group 1
Tue_07:00 -> Group 60
Tue_08:00 -> Group 2

----------------------------------------

Randomized scenarios:

Scenario 1:
Weights matrix:
  Mo07 Mo08 Tu07 Tu08 
[[3.5  2.99 5.39 7.18]
 [2.48 0.5  4.91 0.33]
 [4.24 7.55 4.52 8.12]
 [7.04 6.08 7.88 6.32]
 [7.44 2.37 7.16 2.1 ]
 [4.42 5.53 4.57 8.2 ]]

Matching results:
Mon_07:00 -> Group 1
Mon_08:00 -> Group 5
Tue_07:00 -> Group 3
Tue_08:00 -> Group 2

----------------------------------------

Scenario 2:
Weights matrix:
  Mo07 Mo08 Tu07 Tu08 
[[4.15 4.22 3.25 9.06]
 [0.07 0.8  4.33 2.8 ]
 [2.6  7.38 6.56 8.56]
 [7.02 5.34 5.1  4.75]
 [7.   2.33 6.66 3.79]
 [4.08 2.92 3.22 8.31]]

Matching results:
Mon_07:00 -> Group 2
Mon_08:00 -> Group 5
Tue_07:00 -> Group 60
Tue_08:00 -> Group 4

----------------------------------------

Scenari